In [ ]:
!pip install pymupdf langchain langchain_groq langchain_community faiss-cpu chromadb

In [2]:
!wget https://github.com/krishnaik06/Agentic-LanggraphCrash-course/raw/main/4-Multimodal/multimodal_sample.pdf

--2025-08-04 05:24:11--  https://github.com/krishnaik06/Agentic-LanggraphCrash-course/raw/main/4-Multimodal/multimodal_sample.pdf
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/krishnaik06/Agentic-LanggraphCrash-course/main/4-Multimodal/multimodal_sample.pdf [following]
--2025-08-04 05:24:11--  https://raw.githubusercontent.com/krishnaik06/Agentic-LanggraphCrash-course/main/4-Multimodal/multimodal_sample.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2589 (2.5K) [application/octet-stream]
Saving to: ‘multimodal_sample.pdf.1’

multimodal_sample.p 100%[===================>]   2.53K  --.-

In [6]:
import os
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [7]:
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [8]:
from PIL import Image
import torch

def embed_image(image_data):
  """Embed Image Using CLIP"""
  if isinstance(image_data, str):
    image = Image.open(image_data).convert('RGB')
  else:
    image = image_data

  inputs = processor(images=image, return_tensors="pt", padding=True)

  with torch.no_grad():
    features = model.get_image_features(**inputs)
    features = features / features.norm(dim=-1, keepdim=True)
    return features.squeeze().numpy()

In [9]:
def embed_text(text_data):
  """Embed Text Uisng CLIP"""
  inputs = processor(text=text_data, return_tensors="pt", padding=True, truncation=True, max_length=77)

  with torch.no_grad():
    features = model.get_text_features(**inputs)
    features = features / features.norm(dim=-1, keepdim=True)
    return features.squeeze().numpy()

In [51]:
import fitz
doc = fitz.open('llama2.pdf')

docs = []
embeddings = []
image_data_store = {}

In [52]:
doc

Document('llama2.pdf')

In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [54]:
from langchain_core.documents import Document
import base64
import io

for index, page in enumerate(doc):

  text = page.get_text()
  if text.strip():
    temp_doc = Document(page_content=text, metadata={"page": index, "type": "text"})
    chunks = splitter.split_documents([temp_doc])

    for chunk in chunks:
      embedding = embed_text(chunk.page_content)
      embeddings.append(embedding)
      docs.append(chunk)


    for iindex, image in enumerate(page.get_images(full=True)):
      try:
        xref = image[0]
        base_image = doc.extract_image(xref)
        img_bytes= base_image['image']

        pil_image = Image.open(io.BytesIO(img_bytes)).convert('RGB')

        image_id = f"page_{index}_image_{iindex}"

        buffered = io.BytesIO()
        img_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')
        image_data_store[image_id] = img_base64

        embedding = embed_image(pil_image)
        embeddings.append(embedding)

        image_doc = Document(
            page_content=f"[Image: {image_id}]",
            metadata={"page": index, "type": "image", "image_id": image_id}
        )
        docs.append(image_doc)

      except Exception as e:
        print(f"Error processing image: {e}")

doc.close()

In [55]:
import numpy as np
from langchain_community.vectorstores import FAISS
embedding_array = np.array(embeddings)

vector_store = FAISS.from_embeddings(
    text_embeddings=[(doc.page_content, emb) for doc, emb in zip(docs, embedding_array)],
    embedding=None,
    metadatas=[doc.metadata for doc in docs]
)

In [56]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")

In [57]:
def retrive(query, k=3):
  query_embed = embed_text(query)

  result = vector_store.similarity_search_by_vector(
      embedding=query_embed,
      k=k
  )

  return result

In [58]:
from langchain_core.messages import HumanMessage
def create_multimodal_message(query, retrieved_docs):
    """Create a message with both text and images for GPT-4V."""
    content_string = f"Question: {query}\n\nContext:\n"

    text_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "text"]
    image_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "image"]

    if text_docs:
        text_context = "\n\n".join([
            f"[Page {doc.metadata['page']}]: {doc.page_content}"
            for doc in text_docs
        ])
        content_string += f"Text excerpts:\n{text_context}\n"

    for doc in image_docs:
        image_id = doc.metadata.get("image_id")
        if image_id and image_id in image_data_store:
            content_string += f"\n[Image from page {doc.metadata['page']} with ID: {image_id}]\n"


    content_string += "\n\nPlease answer the question based on the provided text and images."

    return HumanMessage(content=content_string)

In [59]:
def multimodal_pdf_rag_pipeline(query):
    """Main pipeline for multimodal RAG."""
    # Retrieve relevant documents
    context_docs = retrive(query, k=5)

    message = create_multimodal_message(query, context_docs)

    response = llm.invoke([message])

    print(f"\nRetrieved {len(context_docs)} documents:")
    for doc in context_docs:
        doc_type = doc.metadata.get("type", "unknown")
        page = doc.metadata.get("page", "?")
        if doc_type == "text":
            preview = doc.page_content[:100] + "..." if len(doc.page_content) > 100 else doc.page_content
            print(f"  - Text from page {page}: {preview}")
        else:
            print(f"  - Image from page {page}")
    print("\n")

    return response.content

In [66]:
query = "Summarize the Image of page 3 where the the figure is 'Figure 3: Safety human evaluation results for Llama 2-Chat compared to other open-source and closedsource models' and explain each model and its score in chart"
answer = multimodal_pdf_rag_pipeline(query)
print(f"\nQuery: {query}")
print("-" * 50)
print(f"Answer: {answer}")
print("=" * 70)


Retrieved 5 documents:
  - Text from page 3: Figure 3: Safety human evaluation results for Llama 2-Chat compared to other open-source and closed-...
  - Text from page 18: Figure 12: Human evaluation results for Llama 2-Chat models compared to open- and closed-source mode...
  - Text from page 2: Llama 2-Chat models generally perform better than existing open-source models. They also appear to
b...
  - Text from page 17: methodology details in Appendix, Section A.3.7. The following section shows helpfulness results; saf...
  - Text from page 31: SFT (Mix)
SFT (Annotation)
RLHF (V1)
0.0
0.2
0.4
0.6
0.8
1.0
Reward Model Score
RLHF (V2)
Figure 20:...



Query: Summarize the Image of page 3 where the the figure is 'Figure 3: Safety human evaluation results for Llama 2-Chat compared to other open-source and closedsource models' and explain each model and its score in chart
--------------------------------------------------
Answer: Based on the provided text, Figure 3 on page 3 shows the sa

In [67]:
query = "Explain the Image on Page 4 About the figure 'Figure 4: Training of Llama 2-Chat' and explain the process in the Image"
answer = multimodal_pdf_rag_pipeline(query)
print(f"\nQuery: {query}")
print("-" * 50)
print(f"Answer: {answer}")
print("=" * 70)


Retrieved 5 documents:
  - Text from page 4: Figure 4: Training of Llama 2-Chat: This process begins with the pretraining of Llama 2 using public...
  - Text from page 2: compute and human annotation, and is often not transparent or easily reproducible, limiting progress...
  - Text from page 9: whether Llama 2-Chat’s responses are unsafe, e.g., “giving detailed instructions on making a bomb” c...
  - Text from page 11: to access its paired output. For GPT-4, we prompt with a zero-shot question “Choose the best answer ...
  - Text from page 15: or to “act as” some public figure. When we provided such instructions to Llama 2-Chat, the subsequen...



Query: Explain the Image on Page 4 About the figure 'Figure 4: Training of Llama 2-Chat' and explain the process in the Image
--------------------------------------------------
Answer: Based on the provided text, I will explain the image on Page 4, specifically Figure 4: Training of Llama 2-Chat.

The image on Page 4 illustrates the proces